# Introduction to notebook

Insert text

## Installation, Libraries Import & Data Import 📦

### Install Requirements 🎛️

In [1]:
!pip install -r requirements.txt -q

### Importing Libraries 🔌

In [ ]:
# Datahandling
import requests
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns

# Tweet Themes & EDA
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import nltk
nltk.download('stopwords', quiet=True)
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
from scipy.stats import chi2_contingency

# Network analysis
from typing import List, Optional
import json
import matplotlib.patches as mpatches
import networkx as nx
from community import community_louvain
import plotly.graph_objects as go
import random
from collections import defaultdict
import nbformat

# Model prediction
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import classification_report
import tiktoken
import tqdm

# Additional libraries
import umap
import hdbscan


### Data Import 📨